In [27]:
import numpy as np
import pandas as pd
import ipdb
import os


league_list = ["LEC", "LCS", "PCS", "CBLOL", "TCL", "LLA", "LJL"]

def get_winrates(df_2021, df_2022, df_2023, weighting_n):
    """
    Get winrates
    """
    ### Historical WR
    # Group the DataFrame by team_id and count the number of games played
    games_played_2022 = df_2022.groupby('teamid')['result'].count()
    games_played_2021 = df_2021.groupby('teamid')['result'].count()

    # Group the DataFrame by team_id and count the number of wins
    wins_2022 = df_2022.groupby('teamid')['result'].sum()
    wins_2021 = df_2021.groupby('teamid')['result'].sum()

    # Calculate the win rate for each team
    win_rates_in_2022 = (wins_2022 / games_played_2022) * 100
    win_rates_in_2021 = (wins_2021 / games_played_2021) * 100

    tmp_1 = pd.DataFrame(df_2022.groupby('teamid').count().iloc[:, 0])
    tmp_2 = pd.DataFrame(df_2023.groupby('teamid').count().iloc[:, 0])

    historical_wr_2022 = pd.merge(tmp_1, win_rates_in_2021, how='left', left_index=True, right_index=True)
    historical_wr_2023 = pd.merge(tmp_2, win_rates_in_2022, how='left', left_index=True, right_index=True)

    historical_wr_2022.fillna(value=historical_wr_2022['result'].mean(), inplace=True)
    historical_wr_2023.fillna(value=historical_wr_2023['result'].mean(), inplace=True)

    n_games = weighting_n

    ### Dynamic WR
    # 2022
    tmp = df_2022[['date', 'teamid', 'gameid', 'result']].reset_index().set_index(['teamid', 'date']).sort_index()
    games_played_dynamic = pd.DataFrame()

    # Need to adapt, so that the information is as before the game
    games_played_dynamic['total_games'] = tmp.groupby('teamid')['result'].cumcount()
    games_played_dynamic['dynamic_weighting'] = np.clip(0, games_played_dynamic['total_games']/n_games, 1) # dynamic weighting factor
    games_won_dynamic = pd.DataFrame(tmp.groupby('teamid')['result'].shift()).groupby('teamid')['result'].cumsum()

    tmp = pd.merge(tmp, games_played_dynamic,  how='left', left_on=['date','teamid'], right_on = ['date','teamid'])
    tmp = pd.merge(tmp, games_won_dynamic, how='left', left_on=['date','teamid'], right_on = ['date','teamid'])
    tmp['dynamic_wr'] = tmp['result_y'] / tmp['total_games'] * 100

    # If no data available (new teams) replace with average of 50%
    tmp['dynamic_wr'] = tmp['dynamic_wr'].replace([np.inf, -np.inf], np.nan)
    tmp['dynamic_wr'] = tmp['dynamic_wr'].fillna(value=50)

    dynamic_wr_2022 = tmp

    # 2023
    tmp = df_2023[['date', 'teamid', 'gameid', 'result']].reset_index().set_index(['teamid', 'date']).sort_index()
    games_played_dynamic = pd.DataFrame()

    # Need to adapt, so that the information is as before the game
    games_played_dynamic['total_games'] = tmp.groupby('teamid')['result'].cumcount()
    games_played_dynamic['dynamic_weighting'] = np.clip(0, games_played_dynamic['total_games']/n_games, 1) # dynamic weighting factor
    games_won_dynamic = pd.DataFrame(tmp.groupby('teamid')['result'].shift()).groupby('teamid')['result'].cumsum()

    tmp = pd.merge(tmp, games_played_dynamic,  how='left', left_on=['date','teamid'], right_on = ['date','teamid'])
    tmp = pd.merge(tmp, games_won_dynamic, how='left', left_on=['date','teamid'], right_on = ['date','teamid'])
    tmp['dynamic_wr'] = tmp['result_y'] / tmp['total_games'] * 100

    # If no data available (new teams) replace with average of 50%
    tmp['dynamic_wr'] = tmp['dynamic_wr'].replace([np.inf, -np.inf], np.nan)
    tmp['dynamic_wr'] = tmp['dynamic_wr'].fillna(value=50)

    dynamic_wr_2023 = tmp

    wr_2022 = pd.merge(dynamic_wr_2022, historical_wr_2022, how='left', left_on='teamid', right_on='teamid').reset_index()
    wr_2022['mixed_wr'] = wr_2022['dynamic_wr'] * wr_2022['dynamic_weighting'] + wr_2022['result'] * (1 - wr_2022['dynamic_weighting'])


    wr_2023 = pd.merge(dynamic_wr_2023, historical_wr_2023, how='left', left_on='teamid', right_on='teamid').reset_index()
    wr_2023['mixed_wr'] = wr_2023['dynamic_wr'] * wr_2022['dynamic_weighting'] + wr_2023['result'] * (1 - wr_2022['dynamic_weighting'])

    wr = pd.concat([wr_2022, wr_2023], ignore_index=True).set_index(['gameid_x', 'teamid'])['mixed_wr']

    return wr


def check_form(a, b, c):
    if a > b and b > c:
        return 1
    return 0

def get_streak(df_2022, df_2023):
    """
    Get the streak parameter, indicating whether a team won their last 3 games
    """
    # 2022
    tmp = df_2022[['date', 'teamid', 'gameid', 'result']].reset_index().set_index(['teamid', 'date']).sort_index()
    games_won_dynamic = pd.DataFrame(tmp.groupby('teamid')['result'].shift()).groupby('teamid')['result'].cumsum()

    tmp = pd.merge(tmp, games_won_dynamic, how='left', left_on=['date','teamid'], right_on = ['date','teamid'])

    counter_1 = -2
    counter_2 = -1
    output = []

    tmp['tmp'] = tmp['result_y']
    tmp['tmp'] = tmp['tmp'].fillna(value=0)

    for i in range(len(tmp['tmp'])):
        if i < 1:
            output.append(0)
        else:
            output.append(check_form(tmp['tmp'][i], tmp['tmp'][counter_2], tmp['tmp'][counter_1]))
            counter_1 += 1
            counter_2 += 1

    tmp['streak'] = output

    streak_2022 = tmp.reset_index()

    # 2023
    tmp = df_2023[['date', 'teamid', 'gameid', 'result']].reset_index().set_index(['teamid', 'date']).sort_index()
    games_won_dynamic = pd.DataFrame(tmp.groupby('teamid')['result'].shift()).groupby('teamid')['result'].cumsum()

    tmp = pd.merge(tmp, games_won_dynamic, how='left', left_on=['date','teamid'], right_on = ['date','teamid'])

    counter_1 = -1
    counter_2 = 0
    output = []

    tmp['tmp'] = tmp['result_y']
    tmp['tmp'] = tmp['tmp'].fillna(value=0)

    for i in range(len(tmp['tmp'])):
        if i < 1:
            output.append(0)
        else:
            output.append(check_form(tmp['tmp'][i], tmp['tmp'][counter_2], tmp['tmp'][counter_1]))
            counter_1 += 1
            counter_2 += 1

    tmp['streak'] = output

    streak_2023 = tmp.reset_index()

    streak = pd.concat([streak_2022, streak_2023], ignore_index=True).set_index(['gameid', 'teamid'])['streak']

    return streak


def get_gspd(df_2021, df_2022, df_2023, weighting_n):
    """
    Get gspd
    """

    ### Historical gspd
    # 2022
    tmp = pd.DataFrame(df_2022.groupby('teamid').count().iloc[:, 0])
    gspd_in_2021 = df_2021.groupby('teamid')['gspd'].mean() * 100
    historical_gspd_2022 = pd.merge(tmp, gspd_in_2021, how='left', left_index=True, right_index=True)
    historical_gspd_2022.fillna(value=0, inplace=True)

    # 2023
    tmp = pd.DataFrame(df_2023.groupby('teamid').count().iloc[:, 0])
    gspd_in_2022 = df_2022.groupby('teamid')['gspd'].mean() * 100
    historical_gspd_2023 = pd.merge(tmp, gspd_in_2022, how='left', left_index=True, right_index=True)
    historical_gspd_2023.fillna(value=0, inplace=True)

    n_games = weighting_n

    ### Dynamic gspd
    # 2022
    tmp = df_2022[['date', 'teamid', 'gameid', 'gspd']].reset_index().set_index(['teamid', 'date']).sort_index()
    games_played_dynamic = pd.DataFrame()

    # Need to adapt, so that the information is as before the game
    games_played_dynamic['total_games'] = tmp.groupby('teamid')['gspd'].cumcount()
    games_played_dynamic['dynamic_weighting'] = np.clip(0, games_played_dynamic['total_games']/n_games, 1) # dynamic weighting factor
    gspd_dynamic = pd.DataFrame(tmp.groupby('teamid')['gspd'].shift()).groupby('teamid')['gspd'].cumsum()

    tmp = pd.merge(tmp, games_played_dynamic,  how='left', left_on=['date','teamid'], right_on = ['date','teamid'])
    tmp = pd.merge(tmp, gspd_dynamic, how='left', left_on=['date','teamid'], right_on = ['date','teamid'])
    tmp['dynamic_gspd'] = tmp['gspd_y'] / tmp['total_games'] * 100

    tmp.fillna(value=0, inplace=True)
    dynamic_gspd_2022 = tmp

    # 2023
    tmp = df_2023[['date', 'teamid', 'gameid', 'gspd']].reset_index().set_index(['teamid', 'date']).sort_index()
    games_played_dynamic = pd.DataFrame()

    # Need to adapt, so that the information is as before the game
    games_played_dynamic['total_games'] = tmp.groupby('teamid')['gspd'].cumcount()
    games_played_dynamic['dynamic_weighting'] = np.clip(0, games_played_dynamic['total_games']/n_games, 1) # dynamic weighting factor
    gspd_dynamic = pd.DataFrame(tmp.groupby('teamid')['gspd'].shift()).groupby('teamid')['gspd'].cumsum()

    tmp = pd.merge(tmp, games_played_dynamic,  how='left', left_on=['date','teamid'], right_on = ['date','teamid'])
    tmp = pd.merge(tmp, gspd_dynamic, how='left', left_on=['date','teamid'], right_on = ['date','teamid'])
    tmp['dynamic_gspd'] = tmp['gspd_y'] / tmp['total_games'] * 100

    tmp.fillna(value=0, inplace=True)
    dynamic_gspd_2023 = tmp

    gspd_2022 = pd.merge(dynamic_gspd_2022, historical_gspd_2022, how='left', left_on='teamid', right_on='teamid').reset_index()
    gspd_2022['mixed_gspd'] = gspd_2022['dynamic_gspd'] * gspd_2022['dynamic_weighting'] + gspd_2022['gspd'] * (1 - gspd_2022['dynamic_weighting'])

    gspd_2023 = pd.merge(dynamic_gspd_2023, historical_gspd_2023, how='left', left_on='teamid', right_on='teamid').reset_index()
    gspd_2023['mixed_gspd'] = gspd_2023['dynamic_gspd'] * gspd_2022['dynamic_weighting'] + gspd_2023['gspd'] * (1 - gspd_2022['dynamic_weighting'])

    gspd = pd.concat([gspd_2022, gspd_2023], ignore_index=True).set_index(['gameid_x', 'teamid'])['mixed_gspd']

    gspd.replace([np.inf, -np.inf], np.nan, inplace=True)
    gspd = gspd.fillna(method='bfill')

    return gspd


def get_chmp_wr(champions_played, champions_wr):
    """
    Get average champion winrate of the champions played
    """

    champions = champions_played
    champions_wr_overview = champions_wr

    wr_chmp_player_1 = pd.merge(champions.reset_index(), champions_wr_overview, how='left', left_on=['patch', 'player_1'], right_on=['patch', 'name_clean'])["Win%"]
    wr_chmp_player_2 = pd.merge(champions.reset_index(), champions_wr_overview, how='left', left_on=['patch', 'player_2'], right_on=['patch', 'name_clean'])["Win%"]
    wr_chmp_player_3 = pd.merge(champions.reset_index(), champions_wr_overview, how='left', left_on=['patch', 'player_3'], right_on=['patch', 'name_clean'])["Win%"]
    wr_chmp_player_4 = pd.merge(champions.reset_index(), champions_wr_overview, how='left', left_on=['patch', 'player_4'], right_on=['patch', 'name_clean'])["Win%"]
    wr_chmp_player_5 = pd.merge(champions.reset_index(), champions_wr_overview, how='left', left_on=['patch', 'player_5'], right_on=['patch', 'name_clean'])["Win%"]
    wr_chmp_player_6 = pd.merge(champions.reset_index(), champions_wr_overview, how='left', left_on=['patch', 'player_6'], right_on=['patch', 'name_clean'])["Win%"]
    wr_chmp_player_7 = pd.merge(champions.reset_index(), champions_wr_overview, how='left', left_on=['patch', 'player_7'], right_on=['patch', 'name_clean'])["Win%"]
    wr_chmp_player_8 = pd.merge(champions.reset_index(), champions_wr_overview, how='left', left_on=['patch', 'player_8'], right_on=['patch', 'name_clean'])["Win%"]
    wr_chmp_player_9 = pd.merge(champions.reset_index(), champions_wr_overview, how='left', left_on=['patch', 'player_9'], right_on=['patch', 'name_clean'])["Win%"]
    wr_chmp_player_10 = pd.merge(champions.reset_index(), champions_wr_overview, how='left', left_on=['patch', 'player_10'], right_on=['patch', 'name_clean'])["Win%"]

    wr_chmp_player_1 = wr_chmp_player_1.replace([np.nan, 0],50)
    wr_chmp_player_2 = wr_chmp_player_2.replace([np.nan, 0],50)
    wr_chmp_player_3 = wr_chmp_player_3.replace([np.nan, 0],50)
    wr_chmp_player_4 = wr_chmp_player_4.replace([np.nan, 0],50)
    wr_chmp_player_5 = wr_chmp_player_5.replace([np.nan, 0],50)
    wr_chmp_player_6 = wr_chmp_player_6.replace([np.nan, 0],50)
    wr_chmp_player_7 = wr_chmp_player_7.replace([np.nan, 0],50)
    wr_chmp_player_8 = wr_chmp_player_8.replace([np.nan, 0],50)
    wr_chmp_player_9 = wr_chmp_player_9.replace([np.nan, 0],50)
    wr_chmp_player_10 = wr_chmp_player_10.replace([np.nan, 0],50)

    wr_chmp_list = [
    champions.reset_index()['gameid'],
    wr_chmp_player_1,
    wr_chmp_player_2,
    wr_chmp_player_3,
    wr_chmp_player_4,
    wr_chmp_player_5,
    wr_chmp_player_6,
    wr_chmp_player_7,
    wr_chmp_player_8,
    wr_chmp_player_9,
    wr_chmp_player_10,
    ]

    for i, series in enumerate(wr_chmp_list):
        if i == 0:
            chmp_wr = pd.DataFrame(series)
            chmp_wr.columns = ["gameid"]
        else:
            chmp_wr[f"player_{i}"] = series

    chmp_wr['avg_blue_champion_wr'] = (chmp_wr['player_1'] + chmp_wr['player_2'] + chmp_wr['player_3'] + chmp_wr['player_4'] + chmp_wr['player_5'])/5

    chmp_wr['avg_red_champion_wr'] = (chmp_wr['player_6'] + chmp_wr['player_7'] + chmp_wr['player_8'] + chmp_wr['player_9'] + chmp_wr['player_10'])/5

    chmp_wr['chmp_wr_diff'] = chmp_wr['avg_blue_champion_wr'] - chmp_wr['avg_red_champion_wr']

    chmp_wr = chmp_wr[['gameid', 'chmp_wr_diff']]

    return chmp_wr

def filter_league(dataframe, league_list):
    return dataframe.loc[dataframe['league'].isin(league_list)]


In [28]:
def preprocess():
    """
    Preprocess the data
    """

    raw_df_2021 = pd.read_csv('data/2021_LoL_esports_Data.csv', parse_dates=['date'])
    raw_df_2022 = pd.read_csv('data/2022_LoL_esports_Data.csv', parse_dates=['date'])
    raw_df_2023 = pd.read_csv('data/2023_LoL_esports_Data.csv', parse_dates=['date'])
    champ_wr = pd.read_csv('data/all_patches_final.csv')

    final_df = champ_wr

    final_df["Win%"] = final_df["Win %"].str.rstrip('%')

    final_df["Win%"] = pd.to_numeric(final_df["Win%"])

    grouped_df = final_df.groupby(["patch", "Name"])["Win%"].max().reset_index()

    champions_wr_overview = grouped_df

    output = []

    for i in champions_wr_overview["Name"]:
        text = i
        output.append(text[:len(text)//2])

    champions_wr_overview["name_clean"] = output
    champions_wr_overview = champions_wr_overview.drop("Name", axis=1)

    filtered_df_2021 = filter_league(raw_df_2021, league_list)
    filtered_df_2022 = filter_league(raw_df_2022, league_list)
    filtered_df_2023 = filter_league(raw_df_2023, league_list)

    filtered_df_2021 = filtered_df_2021.loc[filtered_df_2021['position'].isin(["team"])]
    filtered_df_2022 = filtered_df_2022.loc[filtered_df_2022['position'].isin(["team"])]
    filtered_df_2023 = filtered_df_2023.loc[filtered_df_2023['position'].isin(["team"])]

    full_data = pd.concat([raw_df_2022, raw_df_2023])

    patch_information = full_data.loc[full_data['participantid'].isin([1])].set_index('gameid')['patch']
    champion_1 = full_data.loc[full_data['participantid'].isin([1])].set_index('gameid')['champion']
    champion_2 = full_data.loc[full_data['participantid'].isin([2])].set_index('gameid')['champion']
    champion_3 = full_data.loc[full_data['participantid'].isin([3])].set_index('gameid')['champion']
    champion_4 = full_data.loc[full_data['participantid'].isin([4])].set_index('gameid')['champion']
    champion_5 = full_data.loc[full_data['participantid'].isin([5])].set_index('gameid')['champion']
    champion_6 = full_data.loc[full_data['participantid'].isin([6])].set_index('gameid')['champion']
    champion_7 = full_data.loc[full_data['participantid'].isin([7])].set_index('gameid')['champion']
    champion_8 = full_data.loc[full_data['participantid'].isin([8])].set_index('gameid')['champion']
    champion_9 = full_data.loc[full_data['participantid'].isin([9])].set_index('gameid')['champion']
    champion_10 = full_data.loc[full_data['participantid'].isin([10])].set_index('gameid')['champion']

    champions_list = [patch_information,champion_1,
                champion_2,
                champion_3,
                champion_4,
                champion_5,
                champion_6,
                champion_7,
                champion_8,
                champion_9,
                champion_10]

    for i, series in enumerate(champions_list):
        if i == 0:
            champions = pd.DataFrame(series)
            champions.columns = ["patch"]
        else:
            champions[f"player_{i}"] = series

    champions = champions.dropna(axis=0)
    champions = champions.replace(['Nunu & Willump'], 'Eskimo')

    winrate_pm = get_winrates(filtered_df_2021, filtered_df_2022, filtered_df_2023, 7)
    gspd_pm = get_gspd(filtered_df_2021, filtered_df_2022, filtered_df_2023, 7)
    streak_pm = get_streak(filtered_df_2022, filtered_df_2023)
    chmp_wr_pm = get_chmp_wr(champions, champions_wr_overview)

    only_blue_2022 = filtered_df_2022.loc[filtered_df_2022['side'].isin(["Blue"])].set_index('gameid')
    only_red_2022 = filtered_df_2022.loc[filtered_df_2022['side'].isin(["Red"])].set_index('gameid')

    only_blue_2023 = filtered_df_2023.loc[filtered_df_2023['side'].isin(["Blue"])].set_index('gameid')
    only_red_2023 = filtered_df_2023.loc[filtered_df_2023['side'].isin(["Red"])].set_index('gameid')

    blue_vs_red_2022 = pd.merge(only_blue_2022['teamid'], only_red_2022['teamid'], how='left', left_index=True, right_index=True)
    blue_vs_red_2023 = pd.merge(only_blue_2023['teamid'], only_red_2023['teamid'], how='left', left_index=True, right_index=True)

    blue_vs_red = pd.concat([blue_vs_red_2022, blue_vs_red_2023]).reset_index()
    blue_results = pd.concat([only_blue_2022[['result']], only_blue_2023[['result']]]).reset_index()

    blue_vs_red = blue_vs_red.dropna(axis=0)
    
    ipdb.set_trace()

    # Merge the blue side data (controlled by 'teamid_x' on the left side)
    model_df = pd.merge(blue_vs_red, winrate_pm, left_on=['gameid', 'teamid_x'], right_on=['gameid_x', 'teamid'], how="left") # wr blue side
    model_df = pd.merge(model_df, streak_pm, left_on=['gameid', 'teamid_x'], right_on=['gameid', 'teamid'], how="left") # streak blue side
    model_df = pd.merge(model_df, gspd_pm, left_on=['gameid', 'teamid_x'], right_on=['gameid_x', 'teamid'], how="left") # gspd blue side

    # Merge the red side data (controlled by 'teamid_y' on the left side)
    model_df = pd.merge(model_df, winrate_pm, left_on=['gameid', 'teamid_y'], right_on=['gameid_x', 'teamid']) # wr red side
    model_df = pd.merge(model_df, streak_pm, left_on=['gameid', 'teamid_y'], right_on=['gameid', 'teamid']) # streak red side
    model_df = pd.merge(model_df, gspd_pm, left_on=['gameid', 'teamid_y'], right_on=['gameid_x', 'teamid']) # gspd red side

    # Merge the results from blue perspective
    model_df = pd.merge(model_df, blue_results, left_on=['gameid'], right_on=['gameid'])

    # Merge the champion wr difference
    model_df = pd.merge(model_df, chmp_wr_pm, left_on='gameid', right_on='gameid')

    # Drop unnecessary columns
    model_df = model_df.drop(['teamid_x', 'teamid_y', 'mixed_wr_x', 'streak_x', 'mixed_gspd_x', 'mixed_wr_y', 'streak_y', 'mixed_gspd_y'], axis=1)
    model_df = model_df.set_index('gameid')

    return model_df

In [29]:
preprocess()

/tmp/ipykernel_17754/2747030273.py:7: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_df_2022 = pd.read_csv('data/2022_LoL_esports_Data.csv', parse_dates=['date'])
/tmp/ipykernel_17754/2747030273.py:8: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_df_2023 = pd.read_csv('data/2023_LoL_esports_Data.csv', parse_dates=['date'])


> /tmp/ipykernel_17754/2747030273.py(95)preprocess()
     94     # Merge the blue side data (controlled by 'teamid_x' on the left side)
---> 95     model_df = pd.merge(blue_vs_red, winrate_pm, left_on=['gameid', 'teamid_x'], right_on=['gameid_x', 'teamid'], how="left") # wr blue side
     96     model_df = pd.merge(model_df, streak_pm, left_on=['gameid', 'teamid_x'], right_on=['gameid', 'teamid'], how="left") # streak blue side

ipdb> n
> /tmp/ipykernel_17754/2747030273.py(96)preprocess()
     95     model_df = pd.merge(blue_vs_red, winrate_pm, left_on=['gameid', 'teamid_x'], right_on=['gameid_x', 'teamid'], how="left") # wr blue side
---> 96     model_df = pd.merge(model_df, streak_pm, left_on=['gameid', 'teamid_x'], right_on=['gameid', 'teamid'], how="left") # streak blue side
     97     model_df = pd.merge(model_df, gspd_pm, left_on=['gameid', 'teamid_x'], right_on=['gameid_x', 'teamid'], how="left") # gspd blue side

ipdb> model_df
                     gameid                      

ipdb> model_df
                     gameid                                 teamid_x  \
0     ESPORTSTMNT04_2090326  oe:team:47ae4d96b0991cd0bc4271a5d03ebbf   
1     ESPORTSTMNT04_2090342  oe:team:cd62e47544f14f1d239afa659b9d243   
2     ESPORTSTMNT01_2701805  oe:team:8f995652631df209ff11c4542abdcaf   
3     ESPORTSTMNT04_2090354  oe:team:128d0cec08b877912eb91f94a1f3295   
4     ESPORTSTMNT04_2090358  oe:team:78f183fa5a7d8ecb22b9ad272c3abd7   
...                     ...                                      ...   
2401  ESPORTSTMNT01_3335201  oe:team:cbfaf0ca2181f0903d2fcceb6c28f3b   
2402  ESPORTSTMNT01_3333294  oe:team:cbfaf0ca2181f0903d2fcceb6c28f3b   
2403  ESPORTSTMNT01_3333302  oe:team:cbfaf0ca2181f0903d2fcceb6c28f3b   
2404  ESPORTSTMNT06_2778084  oe:team:7338408a0fe0217451d2c9a567db999   
2405  ESPORTSTMNT06_2777123  oe:team:128d0cec08b877912eb91f94a1f3295   

                                     teamid_y   mixed_wr  streak  mixed_gspd  
0     oe:team:e8d8ae992fe72acb7adc7bfcee3

ipdb> n
> /tmp/ipykernel_17754/2747030273.py(105)preprocess()
    104     # Merge the results from blue perspective
--> 105     model_df = pd.merge(model_df, blue_results, left_on=['gameid'], right_on=['gameid'])
    106 

ipdb> model_df
                     gameid                                 teamid_x  \
0     ESPORTSTMNT04_2090326  oe:team:47ae4d96b0991cd0bc4271a5d03ebbf   
1     ESPORTSTMNT04_2090342  oe:team:cd62e47544f14f1d239afa659b9d243   
2     ESPORTSTMNT01_2701805  oe:team:8f995652631df209ff11c4542abdcaf   
3     ESPORTSTMNT04_2090354  oe:team:128d0cec08b877912eb91f94a1f3295   
4     ESPORTSTMNT04_2090358  oe:team:78f183fa5a7d8ecb22b9ad272c3abd7   
...                     ...                                      ...   
2401  ESPORTSTMNT01_3335201  oe:team:cbfaf0ca2181f0903d2fcceb6c28f3b   
2402  ESPORTSTMNT01_3333294  oe:team:cbfaf0ca2181f0903d2fcceb6c28f3b   
2403  ESPORTSTMNT01_3333302  oe:team:cbfaf0ca2181f0903d2fcceb6c28f3b   
2404  ESPORTSTMNT06_2778084  oe:team:73384

ipdb> quit


In [26]:
model

,result,chmp_wr_diff
gameid,,
ESPORTSTMNT04_2090326,1,5.494
ESPORTSTMNT04_2090342,1,-0.324
ESPORTSTMNT01_2701805,0,0.942
ESPORTSTMNT04_2090354,0,-2.208
ESPORTSTMNT04_2090358,1,0.066
...,...,...
ESPORTSTMNT01_3335201,0,1.824
ESPORTSTMNT01_3333294,0,1.174
ESPORTSTMNT01_3333302,0,0.638


In [7]:
champions

,patch,player_1,player_2,player_3,player_4,player_5,player_6,player_7,player_8,player_9,player_10
gameid,,,,,,,,,,,
ESPORTSTMNT01_2690210,12.01,Renekton,Xin Zhao,LeBlanc,Samira,Leona,Gragas,Viego,Viktor,Jinx,Alistar
ESPORTSTMNT01_2690219,12.01,Gragas,Lee Sin,Orianna,Jhin,Rakan,Gangplank,Nidalee,Renekton,Syndra,Leona
8401-8401_game_1,12.01,Gwen,Jarvan IV,Syndra,Jinx,Nautilus,Jax,Xin Zhao,Vex,Aphelios,Thresh
ESPORTSTMNT01_2690227,12.01,Renekton,Talon,Zoe,Aphelios,Yuumi,Graves,Xin Zhao,Viktor,Jhin,Leona
8401-8401_game_2,12.01,Rumble,Xin Zhao,Corki,Jinx,Rakan,Gangplank,Lee Sin,Twisted Fate,Ziggs,Leona
...,...,...,...,...,...,...,...,...,...,...,...
ESPORTSTMNT01_3334332,13.05,K'Sante,Viego,Aurelion Sol,Jinx,Thresh,Olaf,Lee Sin,Taliyah,Zeri,Rakan
ESPORTSTMNT06_2778084,13.04,Sett,Wukong,Azir,Varus,Renata Glasc,Olaf,Sejuani,Gragas,Zeri,Rakan
ESPORTSTMNT01_3334356,13.05,Gragas,Vi,Katarina,Kai'Sa,Nautilus,K'Sante,Lee Sin,Taliyah,Jinx,Thresh


In [10]:
winrate_pm = get_winrates(filtered_df_2021, filtered_df_2022, filtered_df_2023, 7)
gspd_pm = get_gspd(filtered_df_2021, filtered_df_2022, filtered_df_2023, 7)
streak_pm = get_streak(filtered_df_2022, filtered_df_2023)
chmp_wr_pm = get_chmp_wr(champions, champions_wr_overview)

In [11]:
winrate_pm

gameid_x               teamid                                 
ESPORTSTMNT03_2541933  oe:team:0916b3c324c8577b0c6344f3e067f62    49.344902
                       oe:team:0916b3c324c8577b0c6344f3e067f62    49.344902
                       oe:team:0916b3c324c8577b0c6344f3e067f62    49.344902
                       oe:team:0916b3c324c8577b0c6344f3e067f62    49.344902
                       oe:team:0916b3c324c8577b0c6344f3e067f62    49.344902
                                                                    ...    
ESPORTSTMNT01_3332787  oe:team:fc8e90107dabb9a35c490b0d86adea0    75.182482
                       oe:team:fc8e90107dabb9a35c490b0d86adea0    75.912409
                       oe:team:fc8e90107dabb9a35c490b0d86adea0    76.642336
                       oe:team:fc8e90107dabb9a35c490b0d86adea0    77.372263
                       oe:team:fc8e90107dabb9a35c490b0d86adea0    78.102190
Name: mixed_wr, Length: 1039392, dtype: float64

In [19]:
import ipdb

def get_winrates(df_2021, df_2022, df_2023, weighting_n):
    """
    Get winrates
    """
    ### Historical WR
    # Group the DataFrame by team_id and count the number of games played
    games_played_2022 = df_2022.groupby('teamid')['result'].count()
    games_played_2021 = df_2021.groupby('teamid')['result'].count()

    # Group the DataFrame by team_id and count the number of wins
    wins_2022 = df_2022.groupby('teamid')['result'].sum()
    wins_2021 = df_2021.groupby('teamid')['result'].sum()

    # Calculate the win rate for each team
    win_rates_in_2022 = (wins_2022 / games_played_2022) * 100
    win_rates_in_2021 = (wins_2021 / games_played_2021) * 100

    tmp_1 = pd.DataFrame(df_2022.groupby('teamid').count().iloc[:, 0])
    tmp_2 = pd.DataFrame(df_2023.groupby('teamid').count().iloc[:, 0])

    historical_wr_2022 = pd.merge(tmp_1, win_rates_in_2021, how='left', left_index=True, right_index=True)
    historical_wr_2023 = pd.merge(tmp_2, win_rates_in_2022, how='left', left_index=True, right_index=True)

    historical_wr_2022.fillna(value=historical_wr_2022['result'].mean(), inplace=True)
    historical_wr_2023.fillna(value=historical_wr_2023['result'].mean(), inplace=True)
    ipdb.set_trace()

    n_games = weighting_n

    ### Dynamic WR
    # 2022
    tmp = df_2022[['date', 'teamid', 'gameid', 'result']].reset_index().set_index(['teamid', 'date']).sort_index()
    games_played_dynamic = pd.DataFrame()

    # Need to adapt, so that the information is as before the game
    games_played_dynamic['total_games'] = tmp.groupby('teamid')['result'].cumcount()
    games_played_dynamic['dynamic_weighting'] = np.clip(0, games_played_dynamic['total_games']/n_games, 1) # dynamic weighting factor
    games_won_dynamic = pd.DataFrame(tmp.groupby('teamid')['result'].shift()).groupby('teamid')['result'].cumsum()

    tmp = pd.merge(tmp, games_played_dynamic,  how='left', left_on=['date','teamid'], right_on = ['date','teamid'])
    tmp = pd.merge(tmp, games_won_dynamic, how='left', left_on=['date','teamid'], right_on = ['date','teamid'])
    tmp['dynamic_wr'] = tmp['result_y'] / tmp['total_games'] * 100

    # If no data available (new teams) replace with average of 50%
    tmp['dynamic_wr'] = tmp['dynamic_wr'].replace([np.inf, -np.inf], np.nan)
    tmp['dynamic_wr'] = tmp['dynamic_wr'].fillna(value=50)

    dynamic_wr_2022 = tmp

    # 2023
    tmp = df_2023[['date', 'teamid', 'gameid', 'result']].reset_index().set_index(['teamid', 'date']).sort_index()
    games_played_dynamic = pd.DataFrame()

    # Need to adapt, so that the information is as before the game
    games_played_dynamic['total_games'] = tmp.groupby('teamid')['result'].cumcount()
    games_played_dynamic['dynamic_weighting'] = np.clip(0, games_played_dynamic['total_games']/n_games, 1) # dynamic weighting factor
    games_won_dynamic = pd.DataFrame(tmp.groupby('teamid')['result'].shift()).groupby('teamid')['result'].cumsum()

    tmp = pd.merge(tmp, games_played_dynamic,  how='left', left_on=['date','teamid'], right_on = ['date','teamid'])
    tmp = pd.merge(tmp, games_won_dynamic, how='left', left_on=['date','teamid'], right_on = ['date','teamid'])
    tmp['dynamic_wr'] = tmp['result_y'] / tmp['total_games'] * 100

    # If no data available (new teams) replace with average of 50%
    tmp['dynamic_wr'] = tmp['dynamic_wr'].replace([np.inf, -np.inf], np.nan)
    tmp['dynamic_wr'] = tmp['dynamic_wr'].fillna(value=50)

    dynamic_wr_2023 = tmp

    wr_2022 = pd.merge(dynamic_wr_2022, historical_wr_2022, how='left', left_on='teamid', right_on='teamid').reset_index()
    wr_2022['mixed_wr'] = wr_2022['dynamic_wr'] * wr_2022['dynamic_weighting'] + wr_2022['result'] * (1 - wr_2022['dynamic_weighting'])


    wr_2023 = pd.merge(dynamic_wr_2023, historical_wr_2023, how='left', left_on='teamid', right_on='teamid').reset_index()
    wr_2023['mixed_wr'] = wr_2023['dynamic_wr'] * wr_2022['dynamic_weighting'] + wr_2023['result'] * (1 - wr_2022['dynamic_weighting'])

    wr = pd.concat([wr_2022, wr_2023], ignore_index=True).set_index(['gameid_x', 'teamid'])['mixed_wr']

    return wr

In [ ]:
test_df_2022 = filter_league(df_2022, league_list)

tmp = test_df_2022[['date', 'teamid', 'gameid', 'result']].reset_index().set_index(['teamid', 'date']).sort_index()
games_played_dynamic = pd.DataFrame()

# Need to adapt, so that the information is as before the game
games_played_dynamic['total_games'] = tmp.groupby('teamid')['result'].cumcount()
games_played_dynamic['dynamic_weighting'] = np.clip(0, games_played_dynamic['total_games']/n_games, 1) # dynamic weighting factor

In [ ]:
games_played_dynamic